<a href="https://colab.research.google.com/github/Edith1100/dw_matrix_road_sign/blob/master/D4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure
from sklearn.metrics import accuracy_score

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [0]:
cd 'drive/My Drive/Colab Notebooks/matrix/matrix three/dw_matrix_road_sign/'

/content/drive/My Drive/Colab Notebooks/matrix/matrix three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('train.p')
test = pd.read_pickle('test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
model = Sequential([
  Conv2D(filters=64, kernel_size=(3,3), activation='relu', input_shape=input_shape),
  Flatten(),
  Dense(num_classes, activation='softmax')   
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 64)        1792      
_________________________________________________________________
flatten (Flatten)            (None, 57600)             0         
_________________________________________________________________
dense (Dense)                (None, 43)                2476843   
Total params: 2,478,635
Trainable params: 2,478,635
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])
model.fit(X_train, y_train)

1088/1088 [==============================] - 4s 3ms/step - loss: 9.1727 - accuracy: 0.7703


In [0]:
def get_cnn_v1(input_shape, num_classes):
  return Sequential([
  Conv2D(filters=64, kernel_size=(3,3), activation='relu', input_shape=input_shape),
  Flatten(),
  Dense(num_classes, activation='softmax')   
  ])
  
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy',
                optimizer='Adam',
                metrics=['accuracy'])
  
  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train,
      y_train,
      batch_size = params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs',5),
      verbose = params_fit.get('verbose',1),
      validation_data = params_fit.get('validation_data', (X_train, y_train)),
      callbacks = [tensorboard_callback]
      )
  return model

In [0]:
model = get_cnn_v1(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)

Epoch 1/5
272/272 [==============================] - 3s 12ms/step - loss: 34.8054 - accuracy: 0.7733 - val_loss: 0.1878 - val_accuracy: 0.9564
Epoch 2/5
272/272 [==============================] - 3s 11ms/step - loss: 0.1879 - accuracy: 0.9590 - val_loss: 0.1147 - val_accuracy: 0.9773
Epoch 3/5
272/272 [==============================] - 3s 11ms/step - loss: 0.1304 - accuracy: 0.9723 - val_loss: 0.0797 - val_accuracy: 0.9827
Epoch 4/5
272/272 [==============================] - 3s 11ms/step - loss: 0.1013 - accuracy: 0.9778 - val_loss: 0.0999 - val_accuracy: 0.9785
Epoch 5/5
272/272 [==============================] - 3s 11ms/step - loss: 0.1177 - accuracy: 0.9746 - val_loss: 0.0651 - val_accuracy: 0.9866


In [0]:
def predict(model_trained, X_test, y_test, scoring=accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1)
  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)
  
  return scoring(y_test_norm, y_pred)

In [0]:
predict(model_trained, X_test, y_test)

0.8430839002267574

In [0]:
def train_and_predict(model): 
  model_trained = train_model(model, X_train, y_train)
  return predict(model_trained, X_test, y_test)

In [0]:
def get_cnn_v2(input_shape, num_classes):
  return Sequential([
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
  MaxPool2D(),
  Dropout(0.3),
  
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Flatten(),
  Dense(1024, activation='relu') ,
  Dropout(0.3),
  
  Dense(num_classes, activation='softmax') 

  ])

train_and_predict(get_cnn_v2(input_shape, num_classes))

Epoch 1/5
272/272 [==============================] - 3s 11ms/step - loss: 5.7123 - accuracy: 0.2441 - val_loss: 1.6843 - val_accuracy: 0.5522
Epoch 2/5
272/272 [==============================] - 3s 10ms/step - loss: 1.4438 - accuracy: 0.5723 - val_loss: 0.6692 - val_accuracy: 0.8319
Epoch 3/5
272/272 [==============================] - 3s 10ms/step - loss: 0.7648 - accuracy: 0.7675 - val_loss: 0.2399 - val_accuracy: 0.9484
Epoch 4/5
272/272 [==============================] - 3s 10ms/step - loss: 0.4478 - accuracy: 0.8649 - val_loss: 0.1333 - val_accuracy: 0.9729
Epoch 5/5
272/272 [==============================] - 3s 10ms/step - loss: 0.3132 - accuracy: 0.9050 - val_loss: 0.0933 - val_accuracy: 0.9821


0.8979591836734694

In [0]:
def get_cnn_v3(input_shape, num_classes):
  return Sequential([
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
  Conv2D(filters=32, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),
  
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Flatten(),
  Dense(1024, activation='relu') ,
  Dropout(0.3),
  
  Dense(num_classes, activation='softmax') 

  ])

train_and_predict(get_cnn_v3(input_shape, num_classes))

Epoch 1/5
272/272 [==============================] - 4s 16ms/step - loss: 2.0407 - accuracy: 0.5467 - val_loss: 0.3115 - val_accuracy: 0.9209
Epoch 2/5
272/272 [==============================] - 4s 15ms/step - loss: 0.4394 - accuracy: 0.8701 - val_loss: 0.0984 - val_accuracy: 0.9797
Epoch 3/5
272/272 [==============================] - 4s 15ms/step - loss: 0.2520 - accuracy: 0.9273 - val_loss: 0.0583 - val_accuracy: 0.9881
Epoch 4/5
272/272 [==============================] - 4s 15ms/step - loss: 0.1700 - accuracy: 0.9499 - val_loss: 0.0304 - val_accuracy: 0.9931
Epoch 5/5
272/272 [==============================] - 4s 15ms/step - loss: 0.1455 - accuracy: 0.9574 - val_loss: 0.0249 - val_accuracy: 0.9931


0.9625850340136054

In [0]:
def get_cnn_v4(input_shape, num_classes):
  return Sequential([
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
  MaxPool2D(),
  Dropout(0.3),
  
  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Flatten(),
  Dense(1024, activation='relu') ,
  Dropout(0.3),
  
  Dense(num_classes, activation='softmax') 

  ])

train_and_predict(get_cnn_v4(input_shape, num_classes))

Epoch 1/5
272/272 [==============================] - 5s 19ms/step - loss: 2.2846 - accuracy: 0.3878 - val_loss: 0.5867 - val_accuracy: 0.8443
Epoch 2/5
272/272 [==============================] - 5s 17ms/step - loss: 0.6084 - accuracy: 0.8160 - val_loss: 0.1364 - val_accuracy: 0.9637
Epoch 3/5
272/272 [==============================] - 5s 17ms/step - loss: 0.3118 - accuracy: 0.9079 - val_loss: 0.0825 - val_accuracy: 0.9772
Epoch 4/5
272/272 [==============================] - 5s 17ms/step - loss: 0.2114 - accuracy: 0.9384 - val_loss: 0.0802 - val_accuracy: 0.9792
Epoch 5/5
272/272 [==============================] - 5s 17ms/step - loss: 0.1742 - accuracy: 0.9480 - val_loss: 0.0395 - val_accuracy: 0.9879


0.9585034013605442

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
  MaxPool2D(),
  Dropout(0.3),
  
  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Flatten(),
  Dense(1024, activation='relu') ,
  Dropout(0.3),
  
  Dense(1024, activation='relu') ,
  Dropout(0.3),

  Dense(num_classes, activation='softmax') 

  ])

train_and_predict(get_cnn_v5(input_shape, num_classes))

Epoch 1/5
272/272 [==============================] - 5s 19ms/step - loss: 3.0385 - accuracy: 0.2124 - val_loss: 1.3361 - val_accuracy: 0.5723
Epoch 2/5
272/272 [==============================] - 5s 17ms/step - loss: 1.0933 - accuracy: 0.6531 - val_loss: 0.3193 - val_accuracy: 0.9091
Epoch 3/5
272/272 [==============================] - 5s 18ms/step - loss: 0.4723 - accuracy: 0.8524 - val_loss: 0.1493 - val_accuracy: 0.9557
Epoch 4/5
272/272 [==============================] - 5s 18ms/step - loss: 0.2904 - accuracy: 0.9109 - val_loss: 0.0552 - val_accuracy: 0.9842
Epoch 5/5
272/272 [==============================] - 5s 18ms/step - loss: 0.2144 - accuracy: 0.9355 - val_loss: 0.0426 - val_accuracy: 0.9870


0.9639455782312926

In [0]:
X_train_gray = color.rgb2gray(X_train).reshape(-1,32,32,1)
X_test_gray = color.rgb2gray(X_test).reshape(-1,32,32,1)

In [0]:
model = get_cnn_v5((32,32,1), num_classes)
model_trained = train_model(model, X_train_gray, y_train, params_fit={})
predict(model_trained, X_test_gray, y_test)

Epoch 1/5
272/272 [==============================] - 5s 20ms/step - loss: 2.3952 - accuracy: 0.3185 - val_loss: 0.6121 - val_accuracy: 0.7917
Epoch 2/5
272/272 [==============================] - 5s 18ms/step - loss: 0.5484 - accuracy: 0.8189 - val_loss: 0.1397 - val_accuracy: 0.9603
Epoch 3/5
272/272 [==============================] - 5s 18ms/step - loss: 0.2515 - accuracy: 0.9203 - val_loss: 0.0736 - val_accuracy: 0.9784
Epoch 4/5
272/272 [==============================] - 5s 18ms/step - loss: 0.1563 - accuracy: 0.9497 - val_loss: 0.0346 - val_accuracy: 0.9909
Epoch 5/5
272/272 [==============================] - 5s 18ms/step - loss: 0.1180 - accuracy: 0.9641 - val_loss: 0.0287 - val_accuracy: 0.9913


0.9702947845804989